In [1]:
import requests

In [7]:
import requests # HTTP 요청을 쉽게 보내주는 라이브러리

# 1. API 기본 정보 설정
ENDPOINT_URL = "http://api.data.go.kr/openapi/tn_pubr_public_nutri_food_info_api" # 기술문서에 있는 주소
MY_SERVICE_KEY = "8643b406f397f1f29cb81e5050438222e1d4bf8ff57efe0defedcc155da957a8" # 너의 마이페이지에서 확인 가능

# 2. 요청 파라미터(조건) 설정
params = {
    'serviceKey': MY_SERVICE_KEY,
    'pageNo': 1,      # 첫 번째 페이지
    'numOfRows': 100, # 100개씩 가져오기
    'dataType': 'JSON' # JSON을 지원한다면 이렇게 요청할 수도 있어.
}

# 3. GET 요청 보내기
response = requests.get(ENDPOINT_URL, params=params)

# 4. 응답 확인
if response.status_code == 200:
    print("성공적으로 데이터를 가져왔어! 👍")
    raw_data = response.text # 응답으로 받은 XML/JSON 텍스트 데이터
else:
    print(f"API 호출 실패. 상태 코드: {response.status_code}")
    # 인증키 오류(403)나 다른 오류 코드에 대한 처리 로직이 필요해.

성공적으로 데이터를 가져왔어! 👍


In [8]:
# 4. 응답 확인 (여기를 수정하는 거야!)
if response.status_code == 200:
    print("성공적으로 데이터를 가져왔어! 👍")
    raw_data = response.text # 응답으로 받은 XML/JSON 텍스트 데이터

    # 🛠️ 추가: raw_data를 출력해서 눈으로 확인
    print("--- 가져온 데이터 (Raw Data) ---")
    print(raw_data) 
    print("---------------------------------")

성공적으로 데이터를 가져왔어! 👍
--- 가져온 데이터 (Raw Data) ---
<?xml version="1.0" encoding="UTF-8"?>
<response><header><resultCode>30</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>

---------------------------------


In [6]:
import requests

url = 'http://api.data.go.kr/openapi/tn_pubr_public_nutri_food_info_api'
params ={'serviceKey' : '8643b406f397f1f29cb81e5050438222e1d4bf8ff57efe0defedcc155da957a8', 'pageNo' : '1', 'numOfRows' : '100', 'type' : 'xml', 'foodCd' : '', 'foodNm' : '', 'dataCd' : '', 'typeNm' : '', 'foodOriginCd' : '', 'foodOriginNm' : '', 'foodLv3Cd' : '', 'foodLv3Nm' : '', 'foodLv4Cd' : '', 'foodLv4Nm' : '', 'foodLv5Cd' : '', 'foodLv5Nm' : '', 'foodLv6Cd' : '', 'foodLv6Nm' : '', 'foodLv7Cd' : '', 'foodLv7Nm' : '', 'nutConSrtrQua' : '', 'enerc' : '', 'water' : '', 'prot' : '', 'fatce' : '', 'ash' : '', 'chocdf' : '', 'sugar' : '', 'fibtg' : '', 'ca' : '', 'fe' : '', 'p' : '', 'k' : '', 'nat' : '', 'vitaRae' : '', 'retol' : '', 'cartb' : '', 'thia' : '', 'ribf' : '', 'nia' : '', 'vitc' : '', 'vitd' : '', 'chole' : '', 'fasat' : '', 'fatrn' : '', 'srcCd' : '', 'srcNm' : '', 'foodSize' : '', 'restNm' : '', 'dataProdCd' : '', 'dataProdNm' : '', 'crtYmd' : '', 'crtrYmd' : '', 'instt_code' : '', 'instt_nm' : '' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>30</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>\r\n'


In [3]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
from pathlib import Path

# === 경로 설정 ===
SRC_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\식품의약품안전처_통합식품영양성분정보(음식)_20250408.csv")
OUT_XLSX = SRC_CSV.with_name("food_ingredients_caffeine_include.xlsx")

# === 컬럼 이름 설정 ===
NAME_COL = "식품명"           # B열 식품명
MAJOR_COL = "식품대분류명"     # 선택(없어도 됨)
MIDDLE_COL = "식품중분류명"    # 선택(없어도 됨)

# === 판정 패턴 ===
POSITIVE_PATTERNS = [
    re.compile(r"녹차|말차|홍차|우롱|보이차|마테|밀크티|티라떼|그린티|블랙티", re.I),
    re.compile(r"초코|초콜릿|코코아|카카오", re.I),
    re.compile(r"콜라(?!겔|맛사탕)", re.I),
    re.compile(r"에너지\s*드링크|에너지\s*음료|몬스터|레드불|핫식스", re.I),
    re.compile(r"모카|마키아토|마끼아또|아포가토|에스프레소샷|콜드브루", re.I),
    re.compile(r"녹차아이스크림|그린티\s*아이스크림|말차(빙수|케이크|라떼)", re.I),
    re.compile(r"립톤|얼그레이|다질링|잉글리시\s*브렉퍼스트", re.I),
]
NEGATIVE_PATTERNS = [
    re.compile(r"디카페인|decaf|무카페인|카페인\s*제거", re.I),
    re.compile(r"보리차|옥수수차|루이보스", re.I),
]

# === 함수 ===
def has_caffeine(name: str, major: str = "", middle: str = "") -> bool:
    name = "" if pd.isna(name) else str(name)
    major = "" if pd.isna(major) else str(major)
    middle = "" if pd.isna(middle) else str(middle)
    text = f"{name} {major} {middle}"

    # 예외 우선
    if any(p.search(text) for p in NEGATIVE_PATTERNS):
        return False

    # 양성 규칙
    if any(p.search(text) for p in POSITIVE_PATTERNS):
        return True

    return False

# === CSV 로드 (인코딩 자동 시도) ===
df = None
last_err = None
for enc in ["utf-8-sig", "cp949", "euc-kr", "utf-8"]:
    try:
        df = pd.read_csv(SRC_CSV, encoding=enc)
        used_enc = enc
        break
    except Exception as e:
        last_err = e
if df is None:
    raise RuntimeError(f"CSV 읽기 실패: {last_err}")

# 컬럼 존재 확인
for col in [NAME_COL]:
    if col not in df.columns:
        raise KeyError(f"필수 컬럼이 없습니다: {col} (현재 컬럼: {list(df.columns)[:10]}…)")

# 카페인여부 계산
major = df[MAJOR_COL] if MAJOR_COL in df.columns else ""
middle = df[MIDDLE_COL] if MIDDLE_COL in df.columns else ""
df["카페인여부"] = [has_caffeine(n, mj, md) for n, mj, md in zip(df[NAME_COL], major, middle)]
# 필요 시 영문 키도 추가
# df["caffeine"] = df["카페인여부"].astype(bool)

# 저장
with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="foods")

print("✅ 저장 완료:", OUT_XLSX)
print("인코딩:", used_enc)
print("총 행:", len(df))
print("카페인=True 건수:", int(df["카페인여부"].sum()))

✅ 저장 완료: C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_caffeine_include.xlsx
인코딩: utf-8-sig
총 행: 14584
카페인=True 건수: 1584


In [2]:
pip install xlsxwriter

   ---------------------------------------- 0.0/175.3 kB ? eta -:--:--
   ------ -------------------------------- 30.7/175.3 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------  174.1/175.3 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 175.3/175.3 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
from pathlib import Path

# === 경로 설정 ===
SRC_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\식품의약품안전처_통합식품영양성분정보(음식)_20250408.csv")
OUT_XLSX = SRC_CSV.with_name("food_ingredients_dairy_include.xlsx")

# === 컬럼 이름 설정 ===
NAME_COL = "식품명"           # B열 식품명
MAJOR_COL = "식품대분류명"     # H열 식품대분류명
MIDDLE_COL = "식품중분류명"    # 선택(없어도 됨)

# === 판정 패턴 ===
POSITIVE_PATTERNS = [
    re.compile(r"우유|유유|밀크|연유|분유|탈지분유", re.I),
    re.compile(r"크림|생크림|휘핑크림|크림치즈|리코타|마스카르포네", re.I),
    re.compile(r"치즈|모짜렐라|체다|고다|파마산|프로볼로네", re.I),
    re.compile(r"버터|기버터|ghee", re.I),
    re.compile(r"요구르트|요거트|발효유|유산균음료|야쿠르트", re.I),
    re.compile(r"라떼|밀크티|쉐이크|요거트스무디", re.I),
    re.compile(r"아이스크림|젤라토|크림파스타", re.I),
]

# 예외 (False)
NEGATIVE_PATTERNS = [
    re.compile(r"두유|아몬드밀크|오트밀크|코코넛밀크|마카다미아밀크|캐슈밀크", re.I),
    re.compile(r"비건|non[-\s]?dairy|논데어리", re.I),
    re.compile(r"식물성\s*(크림|치즈|유)"),
    re.compile(r"프림|커피프림", re.I),
]

# 대분류 기반으로 바로 True로 둘 항목
DAIRY_MAJOR = ["유제품류"]

# === 판정 함수 ===
def is_dairy(name: str, major: str = "", middle: str = "") -> bool:
    name = "" if pd.isna(name) else str(name)
    major = "" if pd.isna(major) else str(major)
    middle = "" if pd.isna(middle) else str(middle)
    text = f"{name} {major} {middle}"

    # 1) 대분류 우선
    if any(d in major for d in DAIRY_MAJOR):
        return True

    # 2) 예외 먼저 (식물성 등)
    if any(p.search(text) for p in NEGATIVE_PATTERNS):
        return False

    # 3) 이름/중분류 기반 양성
    if any(p.search(text) for p in POSITIVE_PATTERNS):
        # 단, '맛'만 표기된 경우 제외 (ex. 치즈맛 과자)
        if re.search(r"맛", name) and not re.search(r"치즈케이크|크림빵|버터크림|생크림케이크", name):
            return False
        return True

    return False

# === CSV 로드 ===
df = None
for enc in ["utf-8-sig", "cp949", "euc-kr", "utf-8"]:
    try:
        df = pd.read_csv(SRC_CSV, encoding=enc)
        used_enc = enc
        break
    except Exception as e:
        last_err = e
if df is None:
    raise RuntimeError(f"CSV 읽기 실패: {last_err}")

# === 유제품 여부 계산 ===
major = df[MAJOR_COL] if MAJOR_COL in df.columns else ""
middle = df[MIDDLE_COL] if MIDDLE_COL in df.columns else ""
df["유제품여부"] = [is_dairy(n, mj, md) for n, mj, md in zip(df[NAME_COL], major, middle)]

# === 엑셀로 저장 ===
with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="foods")

print(f"✅ 저장 완료: {OUT_XLSX}")
print(f"인코딩: {used_enc}")
print(f"총 행: {len(df)}")
print(f"유제품=True 건수: {int(df['유제품여부'].sum())}")


✅ 저장 완료: C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_dairy_include.xlsx
인코딩: utf-8-sig
총 행: 14584
유제품=True 건수: 4052


In [5]:
# -*- coding: utf-8 -*-
# 자몽/쿠마린 관련 식품 True, 그 외 False로 분류 → 엑셀 저장

import re
import pandas as pd
from pathlib import Path

# === 경로 설정 (네 PC 경로로 그대로 사용) ===
SRC_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\식품의약품안전처_통합식품영양성분정보(음식)_20250408.csv")
OUT_XLSX = SRC_CSV.with_name("food_ingredients_coumarin_grapefruit_include.xlsx")

# === 컬럼명 (다르면 바꿔줘) ===
NAME_COL   = "식품명"        # B열
MAJOR_COL  = "식품대분류명"  # 선택
MIDDLE_COL = "식품중분류명"  # 선택

# === 포함( True ) 규칙: 자몽/쿠마린 관련 키워드 전부 True (향/맛도 포함) ===
INCLUDE_PATTERNS = [
    # 자몽/그레이프프루트/가공(주스·에이드·청 등)
    re.compile(r"자몽|grapefruit|그레이프프루트|자몽주스|자몽에이드|자몽청|자몽드레싱|citrus\s*paradisi", re.I),
    # 쿠마린 관련 허브/향신료/식물
    re.compile(r"계피|시나몬|cinnamon", re.I),
    re.compile(r"파슬리|parsley", re.I),
    re.compile(r"후추|pepper", re.I),
    re.compile(r"감초|licorice", re.I),
    re.compile(r"타임|thyme|로즈마리|rosemary|라벤더|lavender|캐모마일|chamomile|허브", re.I),
    # 그 외 텍스트에서 자주 보이는 표기
    re.compile(r"양파|onion", re.I),
    re.compile(r"넥타|nectar", re.I),  # 과즙음료 네이밍(넥타)
]

# === 제외( False ) 규칙: 이름은 비슷하지만 무관한 단어 ===
EXCLUDE_PATTERNS = [
    re.compile(r"자두|플럼|plum", re.I),
    re.compile(r"자일리톨|xylitol", re.I),
    re.compile(r"커리|curry", re.I),
    re.compile(r"고추|chili|chilli|pepper\s*sauce", re.I),  # 고추/칠리(고추 계열은 쿠마린과 무관)
    re.compile(r"참깨|sesame", re.I),
]

def is_coumarin_related(name: str, major: str = "", middle: str = "") -> bool:
    """자몽/쿠마린 관련 키워드가 있으면 True. 단, 명시적 제외어가 있으면 False."""
    name  = "" if pd.isna(name) else str(name)
    major = "" if pd.isna(major) else str(major)
    middle= "" if pd.isna(middle) else str(middle)
    text  = f"{name} {major} {middle}"

    # 1) 제외어가 하나라도 있으면 False
    if any(p.search(text) for p in EXCLUDE_PATTERNS):
        return False
    # 2) 포함어가 하나라도 있으면 True
    if any(p.search(text) for p in INCLUDE_PATTERNS):
        return True
    # 3) 그 외는 False
    return False

# === CSV 로드 (인코딩 자동 시도) ===
df = None
last_err = None
for enc in ["utf-8-sig", "cp949", "euc-kr", "utf-8"]:
    try:
        df = pd.read_csv(SRC_CSV, encoding=enc)
        used_enc = enc
        break
    except Exception as e:
        last_err = e
if df is None:
    raise RuntimeError(f"CSV 읽기 실패: {last_err}")

# 필수 컬럼 체크
if NAME_COL not in df.columns:
    raise KeyError(f"'{NAME_COL}' 컬럼을 찾을 수 없습니다. 현재 열들: {list(df.columns)[:12]} ...")

# === 판정 컬럼 생성 ===
major  = df[MAJOR_COL]  if MAJOR_COL  in df.columns else ""
middle = df[MIDDLE_COL] if MIDDLE_COL in df.columns else ""
df["쿠마린여부"] = [is_coumarin_related(n, mj, md) for n, mj, md in zip(df[NAME_COL], major, middle)]

# (선택) 사람이 보기 쉬운 문자열 컬럼
df["쿠마린_권고"] = df["쿠마린여부"].map(lambda x: "동시 섭취를 피하는 것을 권합니다" if x else "")

# === 엑셀 저장 (xlsxwriter 없으면 openpyxl로 자동 대체) ===
engine = "xlsxwriter"
try:
    with pd.ExcelWriter(OUT_XLSX, engine=engine) as writer:
        df.to_excel(writer, index=False, sheet_name="foods")
except Exception:
    engine = "openpyxl"
    with pd.ExcelWriter(OUT_XLSX, engine=engine) as writer:
        df.to_excel(writer, index=False, sheet_name="foods")

print(f"✅ 저장 완료: {OUT_XLSX}")
print(f"인코딩: {used_enc}")
print(f"총 행: {len(df)}")
print(f"쿠마린여부=True 건수: {int(df['쿠마린여부'].sum())}")


✅ 저장 완료: C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_coumarin_grapefruit_include.xlsx
인코딩: utf-8-sig
총 행: 14584
쿠마린여부=True 건수: 350


In [7]:
# -*- coding: utf-8 -*-
import re
import pandas as pd
from pathlib import Path

# === 경로 ===
SRC_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\식품의약품안전처_통합식품영양성분정보(음식)_20250408.csv")
OUT_XLSX = SRC_CSV.with_name("food_ingredients_soy_peanut_include.xlsx")

# === 컬럼명 ===
NAME_COL   = "식품명"
MAJOR_COL  = "식품대분류명"   # 선택
MIDDLE_COL = "식품중분류명"   # 선택

# === 포함(True) 키워드: '단어/재료명' 위주로 구성 ===
SOY_PATTERNS = [
    r"대두", r"두유", r"두부", r"유부", r"비지",
    r"메주", r"된장", r"간장", r"청국장", r"(낫토|나토)", r"템페",
    r"(미소\s*된장|미소)", r"(에다마메|edamame)",
    r"(대두유|콩기름|soy\s*oil)", r"(콩단백|식물성\s*단백|콩고기)",
    r"(대두\s*레시틴|soy\s*lecithin)"
]
PEANUT_PATTERNS = [
    r"땅콩", r"(피너츠|피넛|peanut)s?", r"(땅콩|피넛)\s*버터",
    r"(땅콩|피넛)\s*(분태|분말|가루)", r"(땅콩유|peanut\s*oil)"
]

# === 제외(False) 키워드: 오탐 방지 ===
EXCLUDE_PATTERNS = [
    # '콩' 글자 포함 오탐 방지: 홍콩, 공항코드 등 → 여기선 명시 패턴만 쓰므로 기본적으로 불필요하지만 안전망으로 둠
    r"홍콩", r"말콩", r"원콩", 
    # 다른 견과/씨앗
    r"(참깨|참기름|sesame)", r"(호두|walnut)", r"(아몬드|almond)",
    r"(캐슈|cashew)", r"(피스타치오|pistachio)", r"(헤이즐넛|hazelnut)"
]

SOY_RE   = [re.compile(p, re.I) for p in SOY_PATTERNS]
PEANUT_RE= [re.compile(p, re.I) for p in PEANUT_PATTERNS]
EXC_RE   = [re.compile(p, re.I) for p in EXCLUDE_PATTERNS]

def contains_any(patterns, text: str) -> bool:
    return any(p.search(text) for p in patterns)

def is_soy_or_peanut(name: str, major: str = "", middle: str = "") -> bool:
    name  = "" if pd.isna(name) else str(name)
    major = "" if pd.isna(major) else str(major)
    middle= "" if pd.isna(middle) else str(middle)
    text  = f"{name} {major} {middle}"

    # 제외 먼저
    if contains_any(EXC_RE, text):
        # 단, '참깨+땅콩' 같이 동시 포함 시 땅콩을 우선시하려면 아래 주석 해제
        # if contains_any(PEANUT_RE + SOY_RE, text):
        #     return True
        return False

    # 포함
    if contains_any(SOY_RE, text) or contains_any(PEANUT_RE, text):
        return True

    return False

# === CSV 로드 ===
df = None
last_err = None
for enc in ["utf-8-sig", "cp949", "euc-kr", "utf-8"]:
    try:
        df = pd.read_csv(SRC_CSV, encoding=enc)
        used_enc = enc
        break
    except Exception as e:
        last_err = e
if df is None:
    raise RuntimeError(f"CSV 읽기 실패: {last_err}")

# 필수 컬럼 체크
if NAME_COL not in df.columns:
    raise KeyError(f"'{NAME_COL}' 컬럼을 찾을 수 없습니다. 현재 컬럼: {list(df.columns)[:12]} ...")

major  = df[MAJOR_COL]  if MAJOR_COL  in df.columns else ""
middle = df[MIDDLE_COL] if MIDDLE_COL in df.columns else ""

# === 판정 ===
df["대두_땅콩_여부"] = [is_soy_or_peanut(n, mj, md) for n, mj, md in zip(df[NAME_COL], major, middle)]

# (선택) 안내문
df["알레르기_권고"] = df["대두_땅콩_여부"].map(lambda x: "대두/땅콩 성분 가능 → 섭취 주의 권고" if x else "")

# === 저장 (xlsxwriter 없으면 openpyxl로 자동 대체) ===
engine = "xlsxwriter"
try:
    with pd.ExcelWriter(OUT_XLSX, engine=engine) as w:
        df.to_excel(w, index=False, sheet_name="foods")
except Exception:
    with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as w:
        df.to_excel(w, index=False, sheet_name="foods")

print(f"✅ 저장 완료: {OUT_XLSX}")
print(f"인코딩: {used_enc}")
print(f"총 행: {len(df)}")
print(f"대두/땅콩=True 건수: {int(df['대두_땅콩_여부'].sum())}")


✅ 저장 완료: C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_soy_peanut_include.xlsx
인코딩: utf-8-sig
총 행: 14584
대두/땅콩=True 건수: 501
